In [16]:
import numpy as np
import scipy.special
from scipy.integrate import trapezoid

In [17]:
def mathieu_cem(m, q, x):
    return scipy.special.mathieu_cem(m, q, 180/np.pi*x)

In [18]:
fail = 0
    
qs = np.logspace(-3,3,21)
qs = np.hstack([-qs, qs])

N = 1000
v = np.linspace(-np.pi,np.pi, N)

In [19]:
MM = 10

#====================================================
# First test normalization per DLMF 28.2.30

tol = 1e-13

for m in range(MM):
    for i in range(len(qs)):
        q = qs[i]
    
        ce, _ = mathieu_cem(m,q,v)
        s = trapezoid(v,ce*ce)
    
        diff = s - np.pi
    
        if (abs(diff) > tol):
        
            fail = fail+1  


In [ ]:
from itertools import combinations
from scipy.integrate import quad
#====================================================
# Next test orthogonality per DLMF 28.2,31

tol = 1e-10
MM = 5  # Max order to test
for m1, m2 in combinations(range(MM),2):

    print(f'-----------  [m1,m2] = [{m1},{m2}]  -----------\n')
    for i in range(len(qs)):
        q = qs[i]
        
        #ce1 = mathieu_ce(m1,q,v);
        #ce2 = mathieu_ce(m2,q,v);     
        #s = trapz(v,ce1.*ce2);

        # integral is recommended over trapz
        # For some reason, I need to transpose the output of mathieu_ce.      
        ce = lambda m,q,v: mathieu_cem(m,q,v)[0] 
        f = lambda v: ce(m1,q,v)*ce(m2,q,v)
        s = quad(f, -np.pi, np.pi)[0]
        
        diff = s
        print(f's = {s}, err = {diff}\n')
        if (abs(diff) > tol):
            print(f'Error!  [m1,m2] = [{m1},{m2}], q = {q}, diff = {diff}\n')
            fail = fail+1

    


-----------  [m1,m2] = [0,1]  -----------

s = -7.411720558268343e-16, err = -7.411720558268343e-16
 -7.411720558268343e-16 -7.411720558268343e-16
s = -8.839648981991021e-16, err = -8.839648981991021e-16
 -8.839648981991021e-16 -8.839648981991021e-16
s = -8.561817485948777e-16, err = -8.561817485948777e-16
 -8.561817485948777e-16 -8.561817485948777e-16
s = -8.496694685681179e-16, err = -8.496694685681179e-16
 -8.496694685681179e-16 -8.496694685681179e-16
s = -8.574975695697224e-16, err = -8.574975695697224e-16
 -8.574975695697224e-16 -8.574975695697224e-16
s = -8.935786526912907e-16, err = -8.935786526912907e-16
 -8.935786526912907e-16 -8.935786526912907e-16
s = -9.056456797293484e-16, err = -9.056456797293484e-16
 -9.056456797293484e-16 -9.056456797293484e-16
s = -9.733598583917315e-16, err = -9.733598583917315e-16
 -9.733598583917315e-16 -9.733598583917315e-16
s = -9.651477678224707e-16, err = -9.651477678224707e-16
 -9.651477678224707e-16 -9.651477678224707e-16
s = -1.18228823453671